# HAP

The Housing Assistance Payment (HAP) is  assistance to low-income families.
Eligibility depends on household size, state, and income, before and after a series of deductions involving earnings and various costs such as shelter and childcare.

### How earnings affect a one-person household's housing assistance allotments

Consider a single person household in San Francisco, California (using 2022 AMI level) with \$1,000 monthly earned income and \$600 monthly rent. They would be eligible for \$204 per month in HAP allotments. 

In [1]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

sim_emp = IndividualSim(year=2022)
sim_emp.add_person(name="person", employment_income=1000 * 12)
sim_emp.add_spm_unit(
    name="spm_unit", members=["person"], hud_gross_rent=600 * 12, ami = 138550, pha_payment_standard = 2748
)

#PHA payment is for 1-bedroom in 2019, see: https://sfmohcd.org/sites/default/files/Documents/MOH/Asset%20Management/2019%20AMI_RentLimits-HMFA.pdf

print(
    "HAP: ",
    round(sim_emp.calc("housing_assistance")[0] / 12),
)

HAP:  204


### How earnings affect a one-person household's housing assistance allotments

Now, consider a single person household in San Francisco, California (using 2022 AMI level) with \$3,000 monthly earned income but still \$600 monthly rent. They would be eligible for \$515 per month in HAP allotments. 

In [2]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

sim_emp = IndividualSim(year=2022)
sim_emp.add_person(name="person", employment_income=3000 * 12)
sim_emp.add_spm_unit(
    name="spm_unit", members=["person"], hud_gross_rent=600 * 12, ami = 138550, pha_payment_standard = 2748
)

#PHA payment is for 1-bedroom in 2019, see: https://sfmohcd.org/sites/default/files/Documents/MOH/Asset%20Management/2019%20AMI_RentLimits-HMFA.pdf

print(
    "HAP: ",
    round(sim_emp.calc("housing_assistance")[0] / 12),)

HAP:  154


### How disability status affect a person's housing assistance allotments

Now, consider a 1-person household in San Francisco, California (using 2022 AMI level) with that same \$3,000 monthly earned income and \$600 monthly rent. If this tenant is disabled, they deduct $400 from their adjusted income calculation. 

In [3]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

sim_emp = IndividualSim(year=2022)
sim_emp.add_person(name="person", employment_income=2600 * 12)
sim_emp.add_spm_unit(
    name="spm_unit", members=["person"], hud_gross_rent=600 * 12, ami = 138550, pha_payment_standard = 2748
)

print(
    "HAP: ",
    round(sim_emp.calc("housing_assistance")[0] / 12),)

HAP:  164


### How dependents affect a person's housing assistance allotments

Now, consider a 2-person household, one head of household and one dependent, in San Francisco, California (using 2022 AMI level) with that same \$3,000 monthly earned income and same \$600 monthly rent. With one dependent, this household deducts $480 from their adjusted income, making their HAP 

In [4]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

sim_emp = IndividualSim(year=2022)
sim_emp.add_person(name="person", employment_income=2520 * 12)
sim_emp.add_spm_unit(
    name="spm_unit", members=["person"], hud_gross_rent=600 * 12, ami = 138550, pha_payment_standard = 2748
)

print(
    "HAP: ",
    round(sim_emp.calc("housing_assistance")[0] / 12),)

HAP:  166


### What if we're elsewhere in the U.S.? Let's say... Durango, CO

Consider a 1-person household in Durango, Colorado (using 2022 AMI level) with that same \$3,000 monthly earned income and same \$600 monthly rent as our test case in San Francisco. 

In [5]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

sim_emp = IndividualSim(year=2022)
sim_emp.add_person(name="person", employment_income=3000 * 12)
sim_emp.add_spm_unit(
    name="spm_unit", members=["person"], hud_gross_rent=600 * 12, ami = 89300, pha_payment_standard = 2748
)

print(
    "HAP: ",
    round(sim_emp.calc("housing_assistance")[0] / 12),)

HAP:  154


In [6]:
sim_emp.vary("employment_income", max=2500 * 12, step=120)

import plotly.express as px

LABELS = dict(
    employment_income="Monthly employment income",
    dividend_income="Monthly dividend income",
    income="Monthly income",
    income_source="Income source",
    housing_cost="Monthly housing cost",
    hap="Monthly hap allotment",
    allotment="HAP allotment",
)

emp_df_full = pd.DataFrame(
    dict(
        employment_income=sim_emp.calc("employment_income")[0],
        hud_hap=sim_emp.calc("hud_hap")[0],
        mtr=-sim_emp.deriv("hud_hap", "employment_income"),
        housing_assistance=sim_emp.calc("housing_assistance")[0]
    ))

fig = px.line(
    emp_df_full,
    "employment_income",
    "housing_assistance",
    labels=LABELS,
    title="HAP allotment for a one-person household in California with $600 monthly housing costs",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

In [36]:
def make_graph(x, title, vary_max): 
    sim_emp = IndividualSim(year=2022)
    sim_emp.add_person(name="person", employment_income=2520 * 12)
    sim_emp.add_spm_unit(
        name="spm_unit", members=["person"], hud_gross_rent=600*12, ami = 138550, pha_payment_standard = 2748, receives_housing_assistance = True 
    )
    sim_emp.vary(x, max=vary_max)
    emp_df_full = pd.DataFrame(
        dict(
            x=sim_emp.calc(x)[0],
            hud_hap=sim_emp.calc("hud_hap")[0],
            mtr=-sim_emp.deriv("hud_hap", x),
            housing_assistance=sim_emp.calc("housing_assistance")[0]
    )).rename(columns={"x":x})
    fig = px.line(
        emp_df_full,
        x,
        "housing_assistance",
        labels=LABELS,
        title=title,
    )
    fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
    fig.show()

In [37]:
make_graph("employment_income", "Housing assistance by employment income", 100000)